# Coefficient (beta) difference

In [ ]:
chr=22
for(chr in c(1:23)){
    male_file=paste0("/share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic/final/rbc.males.1000G_p3.chr",
                chr,".osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01")
    female_file=paste0("/share/storage/REDS-III/GWA_G_by_Sex/rbc_female_osmotic/final/rbc.females.1000G_p3.chr",
                  chr,".osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01")
    male_res <- read.table(male_file,header=T,stringsAsFactors=F)
    female_res <- read.table(female_file,header=T,stringsAsFactors=F)

    vars <- c("name","position","A1","A2","MAF","Rsq","n","beta_SNP_add","sebeta_SNP_add")

    all <- merge(male_res[,vars],female_res[,vars],by="name")
    all$z <- (all$beta_SNP_add.x-all$beta_SNP_add.y)/sqrt(all$sebeta_SNP_add.x^2+all$sebeta_SNP_add.y^2)
    all$p <- 2*pnorm(abs(all$z), mean=0,sd=1, lower.tail = FALSE)

    write.table(all,file=paste0("/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis/rbc_male_female_diff_z.chr",
                           chr,".stats"),quote=F,row.names=F)
}


In [ ]:
### START Generate plots ###
basedir=/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis
prefix=rbc_male_female_diff_z
cd $basedir

outFile=$basedir/${prefix}.table
echo -e "VARIANT_ID\tCHR\tPOSITION\tP\tTYPE" > $outFile
for chr in {1..23}; do
    inFile=$basedir/${prefix}.chr${chr}.stats
    echo Processing $inFile
    awk -v CHR=$chr 'NR>1{if (($3 eq "A" || $3 eq "C" || $3 eq "G" || $3 eq "T") && ($4 eq "A" || $4 eq "C" || $4 eq "G" || $4 eq "T")) {
                        print $1"\t"CHR"\t"$2"\t"$19"\tsnp";
                    } else {
                        print $1"\t"CHR"\t"$2"\t"$19"\tindel";
                    }}' $inFile  >> $outFile
done




In [ ]:
/share/storage/REDS-III/scripts/qsub_job.sh \
    --job_name gwas_plots \
    --script_prefix $basedir/${prefix}.plot \
    --mem 10 \
    --cpu 1 \
    --priority 10 \
    --program /share/storage/REDS-III/common/software/R/generate_gwas_plots.v9.R \
    --in $basedir/${prefix}.table \
    --in_chromosomes autosomal_nonPAR \
    --in_header \
    --out $basedir/${prefix} \
    --col_id VARIANT_ID \
    --col_chromosome CHR \
    --col_position POSITION \
    --col_p P \
    --col_variant_type TYPE \
    --generate_snp_indel_manhattan_plot \
    --manhattan_odd_chr_color red \
    --manhattan_even_chr_color blue \
    --manhattan_points_cex 1.5 \
    --manhattan_cex_axis 2 \
    --manhattan_cex_lab 2 \
    --generate_snp_indel_qq_plot \
    --qq_lines \
    --qq_points_bg black \
    --qq_lambda

### END Generate plots ###



## Generate tables of SNPs with p<0.001

In [ ]:
# p<0.001
basedir=/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis
cd $basedir

outFile=$basedir/rbc_male_female_diff_z.p_lte_0.001
    head -n 1 $basedir/rbc_male_female_diff_z.chr1.stats > $outFile
    for (( chr=1; chr<24; chr++ )); do
        inFile=$basedir/rbc_male_female_diff_z.chr${chr}.stats
        echo Processing $inFile
        tail -n +2 $inFile |
          perl -lane 'if ($F[18] <= 0.001) { print; }' >> $outFile
    done



# Sort
R


base_dir=paste0("/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis")
inData=read.table(paste0(base_dir,"/rbc_male_female_diff_z.p_lte_0.001"),
                      header = TRUE)
inData = inData[order(inData$p),]
write.csv(inData, file=paste0(base_dir,
                    "/rbc_male_female_diff_z.p_lte_0.001.csv"),
                        row.names = FALSE)




# p-value difference

In [ ]:
chr=17
for(chr in c(1:23)){
    male_file=paste0("/share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic/final/rbc.males.1000G_p3.chr",
                chr,".osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01")
    female_file=paste0("/share/storage/REDS-III/GWA_G_by_Sex/rbc_female_osmotic/final/rbc.females.1000G_p3.chr",
                  chr,".osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01")
    male_res <- read.table(male_file,header=T,stringsAsFactors=F)
    female_res <- read.table(female_file,header=T,stringsAsFactors=F)

    vars <- c("name","position","A1","A2","MAF","Rsq","n","beta_SNP_add","sebeta_SNP_add","p")

    all <- merge(male_res[,vars],female_res[,vars],by="name")
    all$p_diff <- unlist(apply(all[,c("p.x","p.y")],1,function(t) abs(t[1]-t[2])/max(t[1],t[2])))
    # p_diff is uniformly distributed U[0,1]
    all$p <- 1-all$p_diff

    write.table(all,file=paste0("/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis/rbc_male_female_diff_pDiff.chr",
                           chr,".stats"),quote=F,row.names=F)
}


In [ ]:
### START Generate plots ###
basedir=/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis
prefix=rbc_male_female_diff_pDiff
cd $basedir

outFile=$basedir/${prefix}.table
echo -e "VARIANT_ID\tCHR\tPOSITION\tP\tTYPE" > $outFile
for chr in {1..23}; do
    inFile=$basedir/${prefix}.chr${chr}.stats
    echo Processing $inFile
    awk -v CHR=$chr 'NR>1{if (($3 eq "A" || $3 eq "C" || $3 eq "G" || $3 eq "T") && ($4 eq "A" || $4 eq "C" || $4 eq "G" || $4 eq "T")) {
                        print $1"\t"CHR"\t"$2"\t"$21"\tsnp";
                    } else {
                        print $1"\t"CHR"\t"$2"\t"$21"\tindel";
                    }}' $inFile  >> $outFile
done




In [ ]:
/share/storage/REDS-III/scripts/qsub_job.sh \
    --job_name gwas_plots \
    --script_prefix $basedir/${prefix}.plot \
    --mem 10 \
    --cpu 1 \
    --priority 0 \
    --program /share/storage/REDS-III/common/software/R/generate_gwas_plots.v9.R \
    --in $basedir/${prefix}.table \
    --in_chromosomes autosomal_nonPAR \
    --in_header \
    --out $basedir/${prefix} \
    --col_id VARIANT_ID \
    --col_chromosome CHR \
    --col_position POSITION \
    --col_p P \
    --col_variant_type TYPE \
    --generate_snp_indel_manhattan_plot \
    --manhattan_odd_chr_color red \
    --manhattan_even_chr_color blue \
    --manhattan_points_cex 1.5 \
    --manhattan_cex_axis 2 \
    --manhattan_cex_lab 2 \
    --generate_snp_indel_qq_plot \
    --qq_lines \
    --qq_points_bg black \
    --qq_lambda

### END Generate plots ###



## Generate tables of SNPs with p<0.001

In [ ]:
# p<0.001
basedir=/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis
cd $basedir

outFile=$basedir/rbc_male_female_diff_pDiff.p_lte_0.001
    head -n 1 $basedir/rbc_male_female_diff_pDiff.chr1.stats > $outFile
    for (( chr=1; chr<24; chr++ )); do
        inFile=$basedir/rbc_male_female_diff_pDiff.chr${chr}.stats
        echo Processing $inFile
        tail -n +2 $inFile |
          perl -lane 'if ($F[20] <= 0.001) { print; }' >> $outFile
    done



# Sort
R


base_dir=paste0("/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis")
inData=read.table(paste0(base_dir,"/rbc_male_female_diff_pDiff.p_lte_0.001"),
                      header = TRUE)
inData = inData[order(inData$p),]
write.csv(inData, file=paste0(base_dir,
                    "/rbc_male_female_diff_pDiff.p_lte_0.001.csv"),
                        row.names = FALSE)




## LocusZoom

In [ ]:
### LocusZoom ###
qsub -I -l mem=10gb,walltime=100:00:00
        
base_dir=/share/storage/REDS-III/GWA_G_by_Sex/male_female_diff_stats/osmotic_hemolysis
cd $base_dir
mkdir LocusZoom
cd LocusZoom

male_table=/share/storage/REDS-III/GWA_G_by_Sex/rbc_male_osmotic/processing/rbc.males.1000G_p3.osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01.table
female_table=/share/storage/REDS-III/GWA_G_by_Sex/rbc_female_osmotic/processing/rbc.females.1000G_p3.osmotic~age+race+donfreq+evs+SNP.maf_gt_0.01.table

# pvalues 

echo -e "MarkerName\tP-value" > rbc.male.osmotic.metal
awk 'NR>1 {print "chr"$2":"$3"\t"$4}' \
    ${male_table} \
    >> rbc.male.osmotic.metal

echo -e "MarkerName\tP-value" > rbc.female.osmotic.metal
awk 'NR>1 {print "chr"$2":"$3"\t"$4}' \
    ${female_table} \
    >> rbc.female.osmotic.metal


for sex in male female;do
    for gene in SPTA1 KCNA6 PAX8 SLC4A1 SUMO1P1 ;do
        /share/storage/REDS-III/common/locuszoom/bin/locuszoom \
            --metal $base_dir/LocusZoom/rbc.${sex}.osmotic.metal \
            --refgene $gene \
            --flank 200Kb \
            --build hg19 \
            --pop EUR \
            --source 1000G_Nov2014 \
            --plotonly \
            --prefix $base_dir/LocusZoom/rbc.${sex}.osmotic.${gene} \
            theme=publication \
                rfrows=3 \
                geneFontSize=1.5 \
                showGenes=TRUE \
                showPartialGenes=TRUE \
                showRecomb=FALSE \
                width=10 \
                height=7 \
                axisSize=1.5 \
                axisTextSize=1.5 \
                axisTextColor=black \
                legend='none'
    done
done
